In [1]:
from env.SourcingEnv import *
import numpy as np
import gurobipy as gp
import pickle as pkl
from env.HelperClasses import *
from sim.sim_functions import *
import itertools

GRB = gp.GRB

In [2]:
filename = "output/msource_value_dic_08-04-2022-10-37-54.pkl"

with open(filename, 'rb') as f:
    output_obj = pkl.load(f)

    value_dic = output_obj["state_value_dic"]
    model_params = output_obj["model_params"]
    sourcingEnv = output_obj["mdp_env"]

    sourcingEnv = SourcingEnv(
        lambda_arrival = model_params['mdp_env_params']['lambda'], # or 10
        procurement_cost_vec = np.array(model_params['mdp_env_params']['procurement_cost_vec']),
        supplier_lead_times_vec = np.array(model_params['mdp_env_params']['supplier_lead_times_vec']),
        on_times = np.array(model_params['mdp_env_params']['on_times']), 
        off_times = np.array(model_params['mdp_env_params']['off_times']))

state_s = list(range(BACKORDER_MAX_LP, MAX_INVEN_LP))
state_backorders = list(itertools.product(range(MAX_INVEN_LP - BACKORDER_MAX_LP), range(sourcingEnv.n_suppliers)))
state_onoff = list(itertools.product(range(2), range(sourcingEnv.n_suppliers)))

possible_state_tuples = list(itertools.product(state_s, state_backorders, state_onoff))
poss_states = [[x[0], np.array(list(x[1])), np.array(list(x[2]))] for x in possible_state_tuples]


action_space_tup = [x for x in itertools.product(*([list(range(ACTION_SIZE_LP))]*sourcingEnv.n_suppliers)) ]

# action_space_tup = list(itertools.product(range(sourcingEnv.action_size), range(sourcingEnv.n_suppliers))) 
action_space = [np.array(list(x)) for x in action_space_tup]

for s in poss_states:
    for i in range(1, len(s)):
        s[i] = list(s[i])

poss_states_set = set([repr(v) for v in poss_states])


In [3]:
# need to write a pij function
# tau = sourcingEnv.compute_event_arrival_time(a)

m = gp.Model("MDP")
x = {}

def add_in_additional_var(state_obj, action):
    if (state_obj.get_nested_list_repr(), repr(list(action))) not in x:
        state_cost = cost_calc(state_obj)
        action_cost = np.sum(np.multiply(sourcingEnv.procurement_cost_vec, action))
        cost = state_cost + action_cost
        x[state_obj.get_nested_list_repr(), repr(list(action))] = m.addVar(obj = cost, name='var_x..' + state_obj.get_nested_list_repr() + ".." + str(action), vtype=GRB.CONTINUOUS) # default lower bound of 0, obj = cost
        m.addConstr(x[state_obj.get_nested_list_repr(), repr(list(action))] >= 0.0)
        return True

for state in poss_states:
    for a in action_space:
        state_rep = MState(state[0], sourcingEnv.n_suppliers, np.array(state[1]), np.array(state[2]))
        add_in_additional_var(state_rep, a)
        
# need to write a pij function
# tau = sourcingEnv.compute_event_arrival_time(a)

poss_states_new = copy.deepcopy(poss_states)
for j_state in poss_states:
    j_state_obj = MState(j_state[0], sourcingEnv.n_suppliers, np.array(j_state[1]), np.array(j_state[2]))   
    poss_i_states_tuples = [] # possible prev. states
    for a_i in action_space:
        event_probs = sourcingEnv.get_event_probs(a_i)
        for k in range(sourcingEnv.n_suppliers):

            # DEMAND_ARRIVAL
            if BACKORDER_MAX_LP < j_state[0] + 1 < MAX_INVEN_LP:
                i_state_supp = copy.deepcopy(j_state[1])
                i_state_supp[k] = np.clip(j_state[1][k] - a_i[k], 0, MAX_INVEN_LP)
                j_state_s = np.clip(j_state[0] + 1, BACKORDER_MAX_LP, MAX_INVEN_LP)
                change_i_state = MState(j_state_s, sourcingEnv.n_suppliers, np.array(i_state_supp), np.array(j_state[2]))
                poss_i_states_tuples.append((a_i, change_i_state, event_probs[0])) # Event DEMAND_ARRIVAL

                if change_i_state.get_nested_list() not in poss_states_new:
                    poss_states_new.append(change_i_state.get_nested_list())
                if (change_i_state.get_nested_list_repr(), repr(list(a_i))) not in x:
                    add_in_additional_var(change_i_state, a_i)

            # SUPPLY_ARRIVAL
            if BACKORDER_MAX_LP < j_state[0] - 1 < MAX_INVEN_LP:
                i_state_supp = copy.deepcopy(j_state[1])
                i_state_supp[k] = np.clip(j_state[1][k] - a_i[k] + 1, 0, MAX_INVEN_LP)
                j_state_s = np.clip(j_state[0] - 1, BACKORDER_MAX_LP, MAX_INVEN_LP)
                change_i_state = MState(j_state_s, sourcingEnv.n_suppliers, np.array(i_state_supp), np.array(j_state[2]))
                index = sourcingEnv.get_event_index_from_event(Event.SUPPLY_ARRIVAL, k)
                poss_i_states_tuples.append((a_i, change_i_state, event_probs[index])) # Event SUPPLY_ARRIVAL
            
                if change_i_state.get_nested_list() not in poss_states_new:
                    poss_states_new.append(change_i_state.get_nested_list())
                if (change_i_state.get_nested_list_repr(), repr(list(a_i))) not in x:
                    add_in_additional_var(change_i_state, a_i)

            i_state_v = copy.deepcopy(j_state[2])
            if j_state[2][k] == 1:
                i_state_v[k] = 0
                v_event = Event.SUPPLIER_ON
            elif j_state[2][k] == 0:
                i_state_v[k] = 1
                v_event = Event.SUPPLIER_OFF

            change_i_state = MState(j_state[0], sourcingEnv.n_suppliers, np.array(j_state[1]), np.array(i_state_v))
            index = sourcingEnv.get_event_index_from_event(v_event, k)
            poss_i_states_tuples.append((a_i, change_i_state, event_probs[index]))

            if change_i_state.get_nested_list() not in poss_states_new:
                poss_states_new.append(change_i_state.get_nested_list())
            if (change_i_state.get_nested_list_repr(), repr(list(a_i))) not in x:
                add_in_additional_var(change_i_state, a_i)                
            
    m.addConstr(gp.quicksum(x[j_state_obj.get_nested_list_repr(), repr(list(a))] for a in action_space) - gp.quicksum(pij*x[state_i.get_nested_list_repr(), repr(list(a_i2))] for (a_i2, state_i, pij) in poss_i_states_tuples) == 0)
poss_states = copy.deepcopy(poss_states_new)

sa_keys = []
for state_i in poss_states:
    for a in action_space:
        sa_keys.append((str(MState(state_i[0], sourcingEnv.n_suppliers, np.array(state_i[1]), np.array(state_i[2])) ), repr(list(a))))

poss_states_objs = [MState(state[0], sourcingEnv.n_suppliers, state[1], state[2]) for state in poss_states]

# for p in poss_states:
#     print(p)

for s in poss_states_objs:
    for a in action_space:
        if (s.get_nested_list_repr(), repr(list(a))) not in x:
            add_in_additional_var(s, a)

m.addConstr(gp.quicksum(sourcingEnv.compute_event_arrival_time(a, state_obj = state_i)*x[state_i.get_nested_list_repr(), repr(list(a))] for state_i in poss_states_objs for a in action_space) == 1)


Set parameter CSQueueTimeout to value 120
Set parameter CSIdleTimeout to value 60
Set parameter ServerTimeout to value 10
Set parameter TokenServer to value "10.162.183.44"


<gurobi.Constr *Awaiting Model Update*>

In [4]:
################################################################

m.setObjective(GRB.MINIMIZE)

m.optimize()
m.write("model_lp_2source.sol")
m.printStats()
m.printAttr('x')
# print("all variables")
# Optimal Policy 
# for state in poss_states:
#     for a in action_space:
#         guro_var = m.getVarByName('var_x..' + repr(state) + ".." + str(a))
#         # if guro_var is not None and guro_var.X > 0:
#         print(guro_var)



Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 20097 rows, 19584 columns and 108436 nonzeros
Model fingerprint: 0xa7fc680a
Coefficient statistics:
  Matrix range     [1e-03, 9e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 19647 rows and 3936 columns
Presolve time: 0.07s
Presolved: 450 rows, 15648 columns, 79500 nonzeros

Ordering time: 0.00s

Barrier performed 0 iterations in 0.08 seconds (0.04 work units)
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Solved in 39 iterations and 0.09 seconds (0.05 work units)
Optimal objective  1.000000000e+00

Statistics for modelMDP_copy:
  Linear constraint matrix    : 20097 Constrs, 19584 Vars, 108436 NZs
  Matrix coefficient range    : [ 0.0012

In [25]:
import json
data = json.loads(m.getJSONSolution())
nz_sol = [x['VarName'] for x in data['Vars']]

In [54]:
ll = list(nz_sol[0].split("..")[1])

[x for x in ll if x not in ["[", "]", " ", ".", "-", ","  ]]

['3', '0', '0', '0', '0']

In [56]:
nz_sol[0]

'var_x..[-3, [0, 0], [0, 0]]..[0 0]'

In [37]:
value_dic

{'[-1, 0, 0, 1, 1]': (-707.4097736604967, 227),
 '[0, 0, 0, 0, 1]': (-16.497378698481644, 2766),
 '[0, 0, 0, 1, 0]': (-529.2291389877955, 3360),
 '[-1, 0, 1, 1, 1]': (-47.193183925775585, 165),
 '[0, 0, 1, 1, 1]': (-363.86782579499464, 182),
 '[0, 0, 1, 0, 1]': (-16.304071428571433, 74),
 '[0, 0, 1, 1, 0]': (-2235.3625167334408, 77),
 '[-1, 0, 2, 1, 1]': (-724.4269975529776, 186),
 '[0, 0, 2, 1, 1]': (-54.19631313900342, 188),
 '[0, 0, 2, 0, 1]': (-17.881545288730013, 115),
 '[0, 0, 2, 1, 0]': (-37.04219595816423, 116),
 '[-1, 0, 3, 1, 1]': (-437.34399221850776, 160),
 '[0, 0, 3, 1, 1]': (-25.106350874717744, 186),
 '[0, 0, 3, 0, 1]': (-17.22561904761905, 118),
 '[0, 0, 3, 1, 0]': (-29.048742881237253, 120),
 '[-1, 0, 4, 1, 1]': (-1155.4912191459177, 163),
 '[0, 0, 4, 1, 1]': (-20.988664256291315, 155),
 '[0, 0, 4, 0, 1]': (-16.57622288372619, 120),
 '[0, 0, 4, 1, 0]': (-19.53655679463816, 63),
 '[-1, 0, 5, 1, 1]': (-21.718582700305262, 168),
 '[0, 0, 5, 1, 1]': (-683.1031338360787, 18

In [36]:
nz_sol

['var_x..[-3, [0, 0], [0, 0]]..[0 0]',
 'var_x..[-3, [0, 0], [0, 1]]..[0 0]',
 'var_x..[-3, [0, 0], [1, 0]]..[0 0]',
 'var_x..[-3, [0, 0], [1, 1]]..[0 0]',
 'var_x..[-3, [5, 0], [0, 0]]..[0 0]',
 'var_x..[-3, [5, 0], [0, 1]]..[0 0]',
 'var_x..[-3, [5, 0], [1, 0]]..[0 0]',
 'var_x..[-3, [5, 0], [1, 1]]..[0 0]',
 'var_x..[-3, [6, 0], [0, 0]]..[0 0]',
 'var_x..[-3, [6, 0], [0, 1]]..[0 0]',
 'var_x..[-3, [6, 0], [1, 0]]..[0 0]',
 'var_x..[-3, [6, 0], [1, 1]]..[0 0]',
 'var_x..[-3, [7, 0], [0, 0]]..[0 0]',
 'var_x..[-3, [7, 0], [0, 1]]..[0 0]',
 'var_x..[-3, [7, 0], [1, 0]]..[0 0]',
 'var_x..[-3, [7, 0], [1, 1]]..[0 0]',
 'var_x..[-2, [0, 0], [0, 0]]..[0 0]',
 'var_x..[-2, [0, 0], [0, 1]]..[0 0]',
 'var_x..[-2, [0, 0], [1, 0]]..[0 0]',
 'var_x..[-2, [0, 0], [1, 1]]..[0 0]',
 'var_x..[-2, [5, 0], [1, 1]]..[0 0]',
 'var_x..[-2, [6, 0], [0, 0]]..[0 0]',
 'var_x..[-2, [6, 0], [1, 0]]..[0 0]',
 'var_x..[-2, [6, 0], [1, 1]]..[0 0]',
 'var_x..[-1, [0, 0], [0, 0]]..[0 0]',
 'var_x..[-1, [0, 0], [0,

In [27]:
data['Vars']

[{'VarName': 'var_x..[-3, [0, 0], [0, 0]]..[0 0]', 'X': 0.008606184579545529},
 {'VarName': 'var_x..[-3, [0, 0], [0, 1]]..[0 0]', 'X': 0.010028595058097351},
 {'VarName': 'var_x..[-3, [0, 0], [1, 0]]..[0 0]', 'X': 0.05014297529048674},
 {'VarName': 'var_x..[-3, [0, 0], [1, 1]]..[0 0]', 'X': 0.060564848586156544},
 {'VarName': 'var_x..[-3, [5, 0], [0, 0]]..[0 0]', 'X': 3.362763802376216e-11},
 {'VarName': 'var_x..[-3, [5, 0], [0, 1]]..[0 0]',
  'X': 1.8831477293306806e-10},
 {'VarName': 'var_x..[-3, [5, 0], [1, 0]]..[0 0]', 'X': 9.415738646653403e-10},
 {'VarName': 'var_x..[-3, [5, 0], [1, 1]]..[0 0]', 'X': 1.054562728425181e-08},
 {'VarName': 'var_x..[-3, [6, 0], [0, 0]]..[0 0]', 'X': 1.209716976157418e-07},
 {'VarName': 'var_x..[-3, [6, 0], [0, 1]]..[0 0]', 'X': 6.847455275223809e-08},
 {'VarName': 'var_x..[-3, [6, 0], [1, 0]]..[0 0]', 'X': 3.38720753324077e-06},
 {'VarName': 'var_x..[-3, [6, 0], [1, 1]]..[0 0]', 'X': 3.834574954125332e-06},
 {'VarName': 'var_x..[-3, [7, 0], [0, 0]]..

In [5]:
# Optimal Policy 
for state in poss_states:
    for a in action_space:
        guro_var = m.getVarByName('x-' + str(state) +"-" + str(a))
        if guro_var is not None and guro_var.X > 0:
            print(guro_var)

In [6]:
m

<gurobi.Model Continuous instance MDP_copy: 20097 constrs, 19584 vars, Parameter changes: CSQueueTimeout=120.0, ServerTimeout=10, TokenServer=10.162.183.44, CSIdleTimeout=60>

In [32]:
import pickle as pkl
from opt.eval_policy import *
from opt.mc_sim import *
import time
from common.variables import *

filename = "output/msource_value_dic_08-04-2022-10-37-54.pkl"

with open(filename, 'rb') as f:
    output_obj = pkl.load(f)

value_dic = output_obj["state_value_dic"]
model_params = output_obj["model_params"]
sourcingEnv = output_obj["mdp_env"]

In [35]:
filename_lp = filename.split("output/")
filename_lp[1]

'msource_value_dic_08-04-2022-10-37-54.pkl'

In [33]:
value_dic

{'[-1, 0, 0, 1, 1]': (-707.4097736604967, 227),
 '[0, 0, 0, 0, 1]': (-16.497378698481644, 2766),
 '[0, 0, 0, 1, 0]': (-529.2291389877955, 3360),
 '[-1, 0, 1, 1, 1]': (-47.193183925775585, 165),
 '[0, 0, 1, 1, 1]': (-363.86782579499464, 182),
 '[0, 0, 1, 0, 1]': (-16.304071428571433, 74),
 '[0, 0, 1, 1, 0]': (-2235.3625167334408, 77),
 '[-1, 0, 2, 1, 1]': (-724.4269975529776, 186),
 '[0, 0, 2, 1, 1]': (-54.19631313900342, 188),
 '[0, 0, 2, 0, 1]': (-17.881545288730013, 115),
 '[0, 0, 2, 1, 0]': (-37.04219595816423, 116),
 '[-1, 0, 3, 1, 1]': (-437.34399221850776, 160),
 '[0, 0, 3, 1, 1]': (-25.106350874717744, 186),
 '[0, 0, 3, 0, 1]': (-17.22561904761905, 118),
 '[0, 0, 3, 1, 0]': (-29.048742881237253, 120),
 '[-1, 0, 4, 1, 1]': (-1155.4912191459177, 163),
 '[0, 0, 4, 1, 1]': (-20.988664256291315, 155),
 '[0, 0, 4, 0, 1]': (-16.57622288372619, 120),
 '[0, 0, 4, 1, 0]': (-19.53655679463816, 63),
 '[-1, 0, 5, 1, 1]': (-21.718582700305262, 168),
 '[0, 0, 5, 1, 1]': (-683.1031338360787, 18